In [1]:
#program to get data from yahoo finance

#

In [2]:
#import packages
from datetime import datetime
import lxml
from lxml import html, etree
import requests
import numpy as np
import pandas as pd
import time
from time import sleep
import string
import yfinance as yf

In [3]:
def safe_divide(x,y): 
    if y == 0:
        result = None 
    else: 
        result = x/y 
    return(result)

In [4]:
#define scraping functions
def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
    }
    return requests.get(url, headers=headers)

In [17]:
#mini-function to get financial statement info from yahoo return in a dataframe
def get_fs(stock):
    print("starting get_fs: "+stock)
    try:
        history_df = pd.DataFrame()
        try:
            history_df=yf.Ticker(stock).history(period="max", interval="1d")
            sleep(0.25)
        except Exception as ex: 
            print("failed to get history")
            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            message = template.format(type(ex).__name__, ex.args)
            print(message)
        try:
            income_page = get_page("https://finance.yahoo.com/quote/"+ stock +"/financials?p="+stock)
            income = html.fromstring(income_page.content)
            sleep(0.25)
#             print(income)
            balance_sheet_page = get_page("https://finance.yahoo.com/quote/"+ stock +"/balance-sheet?p="+stock)
            balance_sheet = html.fromstring(balance_sheet_page.content)
            sleep(0.25)
            cash_flow_page = get_page("https://finance.yahoo.com/quote/"+ stock +"/cash-flow?p="+stock)    
            cash_flow = html.fromstring(cash_flow_page.content)
        except Exception: 
            print("failed to get html")     
        # get income statement data 
        try:
            annual_date_rows =  income.xpath("//div[contains(@class, 'Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg D(ib) Fw(b)')]")
            annual_date_list = ['Year']
            for d in annual_date_rows: 
                date = d.xpath(".//span")[0].text
                annual_date_list.append(date)       
            income_df = []
            #get all income measures 
            income_rows = income.xpath("//div[contains(@class,'D(tbr) fi-row Bgc($hoverBgColor):h')]")
            for i in income_rows:
                income_list = []
                try: 
                    #print(income_metric)
                    income_metric = i.xpath(".//span")[0].text
                    income_list.append(income_metric)
                except Exception:
                    pass 
                vals = i.xpath(".//div[contains(@class,'Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg')]")
                for v in vals:
                    try:
                        income_figure = v.xpath(".//span")[0].text
                    except Exception:
                        income_figure = None 
                    income_list.append(income_figure)
                income_df.append(income_list)    
            income_df = pd.DataFrame(income_df, columns = annual_date_list)
            #print(income_df)
        except Exception as ex: 
            print("failed to get income statement time series : " +stock)
            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            message = template.format(type(ex).__name__, ex.args)
            print(message)
        # get balance sheet data 
        try:
            annual_date_rows =  balance_sheet.xpath("//div[contains(@class, 'Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg D(ib) Fw(b)')]")
            #annual_date_rows =  balance_sheet.xpath("//div[contains(@class, 'Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)')]")
            annual_date_list = ['Year']
            for d in annual_date_rows: 
                #print(d)
                date = d.xpath(".//span")[0].text
                annual_date_list.append(date)       
            bs_df = []
            
            #get balance_sheet measures  
            bs_rows = balance_sheet.xpath("//div[contains(@class,'D(tbr) fi-row Bgc($hoverBgColor):h')]")
            for i in bs_rows:
                bs_list = []
                try:
                    bs_metric = i.xpath(".//span")
                    for b in bs_metric: 
                        b = b.text
#                         print(b)
                        bs_list.append(b)
                    bs_df.append(bs_list)
                except Exception:
                    pass 
            bs_df = pd.DataFrame(bs_df, columns = annual_date_list) 
            #print(bs_df)
        except Exception as ex: 
            print("failed to get balance sheet time series : " +stock)
            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            message = template.format(type(ex).__name__, ex.args)
            print(message)
            
        # get cash flow statement data
        try: 
            annual_date_rows =  cash_flow.xpath("//div[contains(@class, 'Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg D(ib) Fw(b)')]")
            annual_date_list = ['Year']
            for d in annual_date_rows:
                date = d.xpath(".//span")[0].text
                annual_date_list.append(date)             
            cf_df = []
            #get all cash_flow measures  
            cf_rows = cash_flow.xpath("//div[contains(@class,'D(tbr) fi-row Bgc($hoverBgColor):h')]")
            for i in cf_rows:
                cf_list = []
                try:
                    cf_metric = i.xpath(".//span")
                    for c in cf_metric: 
                        c = c.text
                        #print(b)
                        cf_list.append(c)
                    cf_df.append(cf_list)
                except Exception:
                    pass 
            cf_df = pd.DataFrame(cf_df, columns = annual_date_list) 
            #print(cf_df)            
        except Exception as ex: 
            print("failed to get cash flow time series : " +stock)
            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            message = template.format(type(ex).__name__, ex.args)
            print(message)
        final_list = [income_df, bs_df, cf_df,history_df]
        print("get_fs success")
        return(final_list)
    except Exception as ex: 
        print("failed to execute get_fs function: " +stock)
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        pass
        
#get_fs("AAPL")

starting get_fs: AAPL
get_fs success


[                                                 Year          ttm  \
 0                                       Total Revenue  267,981,000   
 1                                     Cost of Revenue  165,854,000   
 2                                        Gross Profit  102,127,000   
 3                                   Operating Expense   36,536,000   
 4                                    Operating Income   65,591,000   
 5           Net Non Operating Interest Income Expense    1,172,000   
 6                                Other Income Expense      328,000   
 7                                       Pretax Income   67,091,000   
 8                                       Tax Provision    9,876,000   
 9                      Net Income Common Stockholders   57,215,000   
 10           Diluted NI Available to Com Stockholders   57,215,000   
 11                                          Basic EPS         None   
 12                                        Diluted EPS         None   
 13   

In [19]:
# calculates finance metrics and info for each stock 
def process_finance(finance_list):
    print("starting process_finance")
    
    income_stmt = finance_list[0].replace(',','', regex=True)
    balance_sht = finance_list[1].replace(',','', regex=True)
    cash_flow = finance_list[2].replace(',','', regex=True)
    
    #print(income_stmt)
    #daily dividend and split data 
    hist = finance_list[3].replace(',','', regex=True)
    #print(hist)
    dates = income_stmt.columns
 #   print(dates)
    first_file_dt = pd.to_datetime(dates[5])
    #print(first_file_dt)
    
    #get number of quarters with dividend in a row
    
    
    #cast all numbers to floats 
    for i in [1,2,3,4,5]:
        income_stmt.iloc[:,i] = pd.to_numeric(income_stmt.iloc[:,i],downcast="float")

    #get historical data
    revenue_0 = float(income_stmt.iloc[0,-1])
    net_income_0 = float(income_stmt.iloc[9,-1])
    assets_0 = float(balance_sht.iloc[0,-1])
    se_0 = float(balance_sht.iloc[4,-1])
    
    roa_0 = safe_divide(net_income_0, assets_0)
    roe_0 = safe_divide(net_income_0, se_0)
    
    #substitute working capital for current_assets
    #current_assets_0 = 
    working_capital_0 = float(balance_sht.iloc[6,-1])
    
    #get current stats
    consRevGrowth = False
    yoy_revGrowth = None
    
    if (income_stmt.iloc[0,2] > income_stmt.iloc[0,3]*1.03 and income_stmt.iloc[0,3] > income_stmt.iloc[0,4]*1.03
        and income_stmt.iloc[0,4] > income_stmt.iloc[0,5]*1.03):
        consRevGrowth = True 
    else: 
        consRevGrowth = False 
        
    yoy_revGrowth = safe_divide(income_stmt.iloc[0,2],income_stmt.iloc[0,3]) -1 
    
    stat_df = pd.DataFrame(columns = ["consRevGrowth", "yoy_revGrowth", "revenue_0", "net_income_0", "assets_0", "se_0", "roa_0", "roe_0"])
    stat_list= [consRevGrowth, yoy_revGrowth, revenue_0, net_income_0, assets_0, se_0, roa_0, roe_0]
    stat_series = pd.Series(stat_list, index = stat_df.columns)
    stat_df = stat_df.append(stat_series, ignore_index = True)
    print("process finance success")
    return(stat_df)

#process_finance(get_fs("AMZN"))

In [21]:
#main scraping function

def get_finance_data(stock): 
    try:
        key_stats_page = get_page("https://finance.yahoo.com/quote/"+ stock +"/key-statistics?p="+stock)
        key_stats = html.fromstring(key_stats_page.content) 
        sleep(0.25)
        summary_page = get_page("https://finance.yahoo.com/quote/"+ stock +"?p="+stock) or get_page("https://finance.yahoo.com/quote/"+stock+"?p="+stock+"&.tsrc=fin-srch")
        summary = html.fromstring(summary_page.content)
        sleep(0.25)
        profile_page = get_page("https://finance.yahoo.com/quote/"+stock+"/profile?p="+stock)
        profile = html.fromstring(profile_page.content)
        analysis_page = get_page("https://finance.yahoo.com/quote/"+stock+"/analysis?p="+stock)
        analysis = html.fromstring(analysis_page.content)
        #stock_info = yf.Ticker(stock).info 
        sleep(0.25)
    except Exception as ex: 
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        print("failed to grab pages: "+stock)
        pass
    
    try:
        sector = profile.xpath("//*[@id='Col1-0-Profile-Proxy']/section/div[1]/div/div/p[2]/span[2]")[0].text
        industry = profile.xpath("//*[@id='Col1-0-Profile-Proxy']/section/div[1]/div/div/p[2]/span[4]")[0].text
    except Exception:
        sector = None
        industry = None
        print("failed to get sector/industry: "+stock)
        pass
    try: 
        price = summary.xpath("//*[@id='quote-summary']/div[1]/table/tbody/tr[1]/td[2]/span")[0].text
        price = float(price)
    except Exception:
        price = None
        print("failed to get price: "+stock)
        pass
    
    try: 
        eps_ttm = summary.xpath("//*[@id='quote-summary']/div[2]/table/tbody/tr[4]/td[2]/span")[0].text
        eps_ttm = float(eps_ttm)
        #print(eps_ttm)
    except Exception: 
        eps_ttm = None
        print("failed to get earnings per share: "+stock)
        pass

    try: 
        pe_ttm = summary.xpath("//*[@id='quote-summary']/div[2]/table/tbody/tr[3]/td[2]/span")[0].text
        if 'k' in pe_ttm:
            pe_ttm = float(pe_ttm.replace("k",""))*1000
        elif 'M' in pe_ttm:
            pe_ttm = float(pe_ttm.replace("M",""))*1000000
        else:
            pe_ttm = float(pe_ttm)
        #float(stock_info['trailingPE'])
    except Exception:
        pe_ttm = float(0)
        print("failed to get pe_ttm: "+stock)
        pass 

#     try: 


#     except Exception:
#         pe_ttm = Nones
#         print("failed to parse pe_ttm: "+stock)
#         pass 

    try: 
        roa_ttm = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[3]/div/div[3]/div/div/table/tbody/tr[1]/td[2]")[0].text 
        roa_ttm = float(roa_ttm.replace("%",""))/100
    except Exception: 
        roa_ttm = None
        print("failed to get roa_ttm: "+stock)
        pass 

    try:
        roe_ttm = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[3]/div/div[3]/div/div/table/tbody/tr[2]/td[2]")[0].text 
        roe_ttm = float(roe_ttm.replace("%",""))/100
    except Exception: 
        roe_ttm  = None
        print("failed to get roe_ttm: "+stock)
        pass 
    
    #get debt to equity ratio
    try:
        debt_equity_ratio_mrq = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[3]/div/div[5]/div/div/table/tbody/tr[4]/td[2]")[0].text
        debt_equity_ratio_mrq = float(debt_equity_ratio_mrq)
    except Exception:
        debt_equity_ratio_mrq = None
        print("failed to get debt_equity_ratio_mrq: "+stock)
        pass
    
    #cast revenue to float 
    try:
        
        revenue_ttm = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[3]/div/div[4]/div/div/table/tbody/tr[1]/td[2]")[0].text 
        if 'M' in revenue_ttm:
            revenue_ttm = float(revenue_ttm.replace("M",""))*1000000
        elif 'B' in revenue_ttm:
            revenue_ttm = float(revenue_ttm.replace("B",""))*1000000000
        else: revenue_ttm = 0
    except Exception:
        revenue_ttm = None
        print("failed to get revenue_ttm: "+stock)
        pass

    try:
        ma_200 = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[2]/div/div[1]/div/div/table/tbody/tr[7]/td[2]")[0].text
        ma_200 = float(ma_200)
    except Exception:
        ma_200 = None
        print("failed to get ma_200: "+stock)
        pass

    try:
        ma_50 = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[2]/div/div[1]/div/div/table/tbody/tr[6]/td[2]")[0].text
        ma_50 = float(ma_50)
    except Exception:
        ma_50 = None
        print("failed to get ma_50: "+stock)
        pass

    try:
        current_ratio_mrq = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[3]/div/div[5]/div/div/table/tbody/tr[5]/td[2]")[0].text
        current_ratio_mrq = float(current_ratio_mrq)
    except Exception:
        current_ratio_mrq = None
        print("failed to get current_ratio_mrq: "+stock)
        pass

    try:
        div_yield_ttm = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[2]/div/div[3]/div/div/table/tbody/tr[4]/td[2]")[0].text
        div_yield_ttm = float(div_yield_ttm.replace("%",""))
    except Exception:
        div_yield_ttm= None
        print("failed to get div_yield_ttm: "+stock)
        pass 

    try:
        div_yield_5y_avg = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[2]/div/div[3]/div/div/table/tbody/tr[5]/td[2]")[0].text
        div_yield_5y_avg = float(div_yield_5y_avg.replace("%","")) 
    except Exception:
        div_yield_5y_avg = None
        print("failed to get div_yield_5y_avg: "+stock)
        pass

    try:
        high_52 = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[2]/div/div[1]/div/div/table/tbody/tr[4]/td[2]")[0].text  
        high_52  = float(high_52 )
    except Exception:
        high_52 = None
        print("failed to get high_52: "+stock)
        pass

    try:
        low_52 = key_stats.xpath("//*[@id='Col1-0-KeyStatistics-Proxy']/section/div[3]/div[2]/div/div[1]/div/div/table/tbody/tr[5]/td[2]")[0].text
        low_52  = float(low_52 )
    except Exception:
        low_52 = None 
        print("failed to get low_52: "+stock)
        pass
    
    try:
#         print("growth")
        growth_est_f5y = float(analysis.xpath("//*[@id='Col1-0-AnalystLeafPage-Proxy']/section/table[6]/tbody/tr[5]/td[2]")[0].text.replace("%", ""))/100
#         print(growth_est_f5y)
        growth_est_p5y = float(analysis.xpath("//*[@id='Col1-0-AnalystLeafPage-Proxy']/section/table[6]/tbody/tr[6]/td[2]")[0].text.replace("%", ""))/100
#         print(growth_est_p5y)
        growth_est_f1y = float(analysis.xpath("//*[@id='Col1-0-AnalystLeafPage-Proxy']/section/table[6]/tbody/tr[4]/td[2]")[0].text.replace("%", ""))/100  
#         print(growth_est_f1y)
#         print(growth_est_f1y)
    except Exception:
        print("failed to annualized growth rates: "+stock)
        growth_est_f5y = None
        growth_est_p5y = None
        growth_est_f1y = None
        
    # get historical/financial statement data
#     try:
#         fs_ts = process_finance(get_fs(stock))
        
#         #print(fs_ts)
#         consRevGrowth_value = fs_ts['consRevGrowth'][0]
#        # print(a)
#         yoy_revGrowth_value = fs_ts['yoy_revGrowth'][0]
#     except Exception:
#         print("failed to get financial statement time series aggregation: "+stock)
    
    #construct data set with finances 
    try: 
        #set column names
        cols = ['ticker','sector', 'industry','price', 'eps_ttm', 'pe_ttm', 'roa_ttm','roe_ttm',
                'debt_equity_ratio_mrq','revenue_ttm', 
                'ma_200', 'ma_50', 'current_ratio_mrq', 
                'div_yield_ttm','div_yield_5y_avg', 'high_52', 'low_52',
#                 'consRevGrowth', 'yoy_revGrowth', 
                'growth_est_f5y', 'growth_est_p5y', 'growth_est_f1y'
               ]
        m_list = [stock, sector, industry,price, eps_ttm, pe_ttm,roa_ttm, roe_ttm, 
                  debt_equity_ratio_mrq, revenue_ttm, 
                  ma_200, ma_50, current_ratio_mrq, 
                  div_yield_ttm,div_yield_5y_avg, high_52, low_52,
#                   consRevGrowth_value, yoy_revGrowth_value, 
                  growth_est_f5y, growth_est_p5y, growth_est_f1y
                 ]
        ss_df = pd.DataFrame([m_list], columns = cols)
        print("Compiled Data for: "+ stock)
        return(ss_df)
    except Exception as ex: 
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        pass   
#get_finance_data("AAL")
#print(finance_df)

In [29]:
#calc predicted value 
# take in a stock and its financial data and try to predict the value of the stock
def predict_value(f, discount_rate):
    eps = f['eps_ttm']
    g = f['growth_est_f5y']
    pe = f['pe_ttm']
    
    #discount rate
    d = discount_rate
    predicted_eps = eps * ((1+g)**5 -1)
    predicted_price = predicted_eps * pe 
    
    #discount predicted price in the future back to the prsent
    current_iv = predicted_price/((1+d)**5 -1)
    return(current_iv)

# finance_df = get_finance_data("AAPL")
# print(predict_value(finance_df,.12))
# finance_df

An exception of type ValueError occurred. Arguments:
('The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().',)


TypeError: 'NoneType' object is not subscriptable

In [10]:
#step to calculate some filters for value
def add_filters(df): 
    #revenue is in thousands
    df['size_filter'] = df['revenue_ttm'] > 1000000000
    df['pe_filter'] =  df['pe_ttm'].apply(lambda x: x> 0 and x < 50 )
    df['div_filter'] = df['div_yield_ttm'] > 0 
    df['strong_div_filter'] = df['div_yield_ttm'] >= 4 
    df['div_above_5y_avg_filter'] = df['div_yield_ttm'] > df['div_yield_5y_avg']
    df['current_ratio_mrq_filter'] = df['current_ratio_mrq'] > 1.2
    df['roa_ttm_filter'] = df['roa_ttm'] > .1 
    return(df)

In [11]:
def get_info(ticker_list):
    initial_screen_list = []
    for i in ticker_list:
        stock = i
        try: 
            fh_ss = get_finance_data(stock)
            #print(fh_ss)
            #add predicted value of stock at a discount rate of 1%
            fh_ss['predicted_intrinsic_value'] = predict_value(fh_ss, .15)
            fh_ss['predicted_premium_percent'] = (safe_divide(fh_ss['price'][0], fh_ss['predicted_intrinsic_value'][0]) - 1)*100
            initial_screen_list.append(fh_ss)
        except Exception:
            pass
        sleep(2)
    try:
        #turn list into df 
        df = pd.concat(initial_screen_list)
        #turn object columns into floats
        df["pe_ttm"] = df.pe_ttm.astype(float)
        df["debt_equity_ratio_mrq"] = df.debt_equity_ratio_mrq.astype(float)     
        df["current_ratio_mrq"] = df.current_ratio_mrq.astype(float)
        df["div_yield_ttm"] = df.div_yield_ttm.astype(float)
        df["div_yield_5y_avg"] = df.div_yield_5y_avg.astype(float)
        
        #add filters 
        df = add_filters(df)
        #print(df)
        return(df)
    
    except Exception:
        print("failed to get info")
        pass